In [19]:
import pandas as pd # library for data analsysis
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import numpy as np # library to handle data in a vectorized manner
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
pd.set_option('display.max_colwidth', None)

In [20]:
ch_ban_imp_merged = pd.read_csv('ch_ban_imp_merged.csv')
bangalore_merged = pd.read_csv("Ban_top_10_venues.csv")
chennai_merged = pd.read_csv("Ch_top_10_venues.csv")

# set number of clusters
kclusters = 5

In [21]:
def create_map(neighborhoods, zoom = 10, cluster_info = False, address=None):
    
    if address == None:
        # create map of Bangalore using latitude and longitude values
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode('Bangalore, India')
        latitude = location.latitude
        longitude = location.longitude
    else:
        location = get_location(address)
        latitude = location.latitude
        longitude = location.longitude

    map_neighborhoods = folium.Map(location=[latitude, longitude], zoom_start=zoom)
    
    if cluster_info:
        
        # set color scheme for the clusters
        x = np.arange(kclusters)
        ys = [i + x + (i*x)**2 for i in range(kclusters)]
        colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
        rainbow = [colors.rgb2hex(i) for i in colors_array]

        # add markers to the map
        markers_colors = []
        for lat, lon, poi, cluster in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Location'], neighborhoods['Cluster Labels']):
            label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
            folium.CircleMarker(
                [lat, lon],
                radius=5,
                popup=label,
                color=rainbow[int(cluster)-1],
                fill=True,
                fill_color=rainbow[int(cluster)-1],
                fill_opacity=0.7).add_to(map_neighborhoods)
    
    else:
        
        # add markers to map
        for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Location']):
            label = '{}, {}'.format(neighborhood, 'Bangalore')
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [float(lat), float(lng)],
                radius=5,
                popup=label,
                color='blue',
                fill=True,
                fill_color='#3186cc',
                fill_opacity=0.7,
                parse_html=False).add_to(map_neighborhoods)

    return map_neighborhoods    

In [22]:
def get_location(address):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    return location

In [24]:
create_map(ch_ban_imp_merged, 12, True, 'Whitefield, Bangalore, India')

In [25]:
create_map(ch_ban_imp_merged, 12, True, 'West mambalam, Chennai, India')

In [28]:
create_map(chennai_merged, 11, True, 'Chennai, India')